<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №1</center>

<center>«Введение в MapReduce модель на Python»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>



<br>
<p style="text-align:right;">Обучающийся: <Вернер К.А.>
<br>6133-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2024</center>


# Введение в MapReduce модель на Python


In [37]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [38]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [39]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [40]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [41]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [42]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [43]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [44]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [45]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [46]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [47]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [48]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [49]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [50]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [51]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(2.193427903531904)),
 (1, np.float64(2.193427903531904)),
 (2, np.float64(2.193427903531904)),
 (3, np.float64(2.193427903531904)),
 (4, np.float64(2.193427903531904))]

## Inverted index 

In [52]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [53]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [54]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [55]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCE and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('banana', 2)])]

## TeraSort

In [56]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.011399083851252101)),
   (None, np.float64(0.0476581184680378)),
   (None, np.float64(0.16264588347527498)),
   (None, np.float64(0.3306067519188547)),
   (None, np.float64(0.35215441516990564)),
   (None, np.float64(0.36645033461077225)),
   (None, np.float64(0.3779598679160018)),
   (None, np.float64(0.3885702262694365)),
   (None, np.float64(0.4597640396088365)),
   (None, np.float64(0.48581597110570707)),
   (None, np.float64(0.4985743908497614))]),
 (1,
  [(None, np.float64(0.521114608033698)),
   (None, np.float64(0.5359210548284016)),
   (None, np.float64(0.5686318762213058)),
   (None, np.float64(0.6108246753252679)),
   (None, np.float64(0.6539430250108614)),
   (None, np.float64(0.6720405023904306)),
   (None, np.float64(0.6854822332221718)),
   (None, np.float64(0.7255394439945665)),
   (None, np.float64(0.7332072869428503)),
   (None, np.float64(0.7359624295654171)),
   (None, np.float64(0.745244023978221)),
   (None, np.float64(0.780250248304797

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [57]:
import numpy as np

# Генерация списка случайных чисел
split = [np.random.randint(30) for _ in range(15)]
print("Сгенерированный список:", split)

# Функция RECORDREADER, которая возвращает список значений
def RECORDREADER():
    return [(0, split[n]) for n in range(len(split))]  # Возвращаем список значений напрямую

# Функция MAP, которая возвращает элементы как кортежи
def MAP(n, spl):
    yield (n, spl)  # Возвращаем кортеж с элементом

# Функция REDUCE, которая находит максимальное значение
def REDUCE(n, spl):
    yield (n, max(spl))  # Возвращаем максимальное значение из списка

# Запуск MapReduce
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print("Максимальное значение:", output)  # Выводим результат

Сгенерированный список: [18, 8, 25, 15, 5, 1, 22, 29, 24, 3, 27, 20, 29, 6, 4]
Максимальное значение: [(0, 29)]


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [58]:
import numpy as np

storage = [np.random.randint(30) for _ in range(15)]
print(storage)

def RECORDREADER():
    return [(0, storage[n]) for n in range(len(storage))]

def MAP(n, stor):
    yield (n, stor)

def REDUCE(n, stor):
    yield (n, sum(stor) / len(stor))

        
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)

[9, 15, 6, 14, 8, 15, 9, 15, 15, 11, 7, 25, 28, 21, 15]
[(0, 14.2)]


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [59]:
def groupbykey(sortied):
  sortied = [(num, num) for num in sorted(sortied)]
  t = {}
  for k1, v1 in sortied:
    t[k1] = t.get(k1, []) + [v1]
  return t.items()
sortied = [1,5,2,2,4,8,8,9,0]
output = groupbykey(sortied)  
print(list(output))

[(0, [0]), (1, [1]), (2, [2, 2]), (4, [4]), (5, [5]), (8, [8, 8]), (9, [9])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [60]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

maps = 3
reducers = 3

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split): #enumerate встроенная функция которая добавляет к счетчик к итерируемому объекту и возвращает его
            yield (userindex, user)
    
    split_size = int(np.ceil(len(input_collection)/ maps)) #Функция ceil округляет в большую сторону числа
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])

def MAP(_, user:str):
    yield (user.id, user)

def REDUCE(user_id: int, users: Iterator[int]):
    yield users[0]

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(partitioned_output)

6 key-value pairs were sent over a network.
[(0, [User(id=0, age=55, social_contacts=20, gender='male'), User(id=3, age=33, social_contacts=800, gender='female')]), (1, [User(id=1, age=25, social_contacts=240, gender='female')]), (2, [User(id=2, age=25, social_contacts=500, gender='female')])]


## Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [61]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row: NamedTuple):
    if row.social_contacts > 300:
        yield (row, row)

def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output
     

[((User(id=2, age=25, social_contacts=500, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')),
  [(User(id=2, age=25, social_contacts=500, gender='female'),
    User(id=2, age=25, social_contacts=500, gender='female'))]),
 ((User(id=3, age=33, social_contacts=800, gender='female'),
   User(id=3, age=33, social_contacts=800, gender='female')),
  [(User(id=3, age=33, social_contacts=800, gender='female'),
    User(id=3, age=33, social_contacts=800, gender='female'))])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [62]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

def MAP(_, rec):
    yield (rec.age, rec.age)

def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[((25, 25), [(25, 25), (25, 25)]),
 ((33, 33), [(33, 33)]),
 ((55, 55), [(55, 55), (55, 55)])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [63]:
input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=33, gender="male", social_contacts=973),
]

def MAP(_, value):
    yield (value, None)

def REDUCE(key, value):
    yield (key, None)

def RECORDREADER():
    return [(u.id, u) for u in input_collection_1 + input_collection_2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[((User(id=0, age=55, social_contacts=20, gender='male'), None), None),
 ((User(id=1, age=25, social_contacts=240, gender='female'), None), None),
 ((User(id=2, age=25, social_contacts=500, gender='female'), None), None),
 ((User(id=3, age=33, social_contacts=800, gender='female'), None), None),
 ((User(id=4, age=33, social_contacts=973, gender='male'), None), None)]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [64]:
def MAP(_, row: NamedTuple):
    yield (row.id, row)

def REDUCE(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows 

def RECORDREADER():
    return [(u.id, u) for u in input_collection_1 + input_collection_2]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[[(2, User(id=2, age=25, social_contacts=500, gender='female')),
  (2, User(id=2, age=25, social_contacts=500, gender='female'))],
 [(3, User(id=3, age=33, social_contacts=800, gender='female')),
  (3, User(id=3, age=33, social_contacts=800, gender='female'))]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [65]:
input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=33, gender="male", social_contacts=973),
]

def RECORDREADER():
    return [(0, a) for a in input_collection_1] + [(0, b) for b in input_collection_2]

def MAP(collection_id, user):
    yield (user, collection_id)

def REDUCE(user, collections):
    if collections == [0]:
        yield user

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [66]:
class Student(NamedTuple):
  id: int
  name: str
  birthday: str
  gender: str
  instituteID: int

class Institute(NamedTuple):
  id: int
  name: str
  size: int

students = [
    Student(1, "Anny", "12.02.2003", "male", 1),
    Student(2, "Peter", "10.11.2002", "female", 1),
    Student(3, "Alex", "24.08.2002", "male", 2),
    Student(4, "Alice", "17.05.2002", "female", 2),
]

institutes = [
    Institute(1, "IIC", 150),
    Institute(2, "FFC", 100),
]

def RECORDREADER():
    return [(student.instituteID, student) for student in students] + [
        (institute.id, institute) for institute in institutes
    ]

def MAP(id: int, row):
    yield (id, row)

def REDUCE(id: int, rows):
    institute = None

    for row in rows:
        if type(row) is Institute:
            institute = row

    for row in rows:
        if type(row) is Student:
            yield (row.id, row.name, row.birthday, row.gender, institute.id, institute.name, institute.size)

list(MapReduce(RECORDREADER, MAP, REDUCE))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(output)

[]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [67]:
def MAP(job_id, user, job):
    yield (job_id, user)

def REDUCE(job_id, rows):
    yield f"job_id = {job_id} has {len(rows)} user(s)"

def RECORDREADER():
    return [(jon_id, user, job) for user, jon_id, job in output]

output1 = MapReduce(RECORDREADER, MAP, REDUCE)
output1 = list(output1)
output

[]

## Вычисление TF-IDF (Term Frequency – Inverse Document Fraquency)

Реализуется в три этапа:

**Этап 1:** Частота слова в документе

**Этап 2:** Количество документов, в которых встречается слово

**Этап 3:** Расчёт TF-IDF

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример документов
documents = [
    "Кошка сидит на ковре",
    "Собака лежит на полу",
    "Кошка и собака играют вместе",
    "Кошка ловит мышь"
]

# Создание объекта TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование документов в матрицу TF-IDF
tfidf_matrix = vectorizer.fit_transform(documents)

# Получение словаря терминов
feature_names = vectorizer.get_feature_names_out()

# Преобразование матрицы в массив
dense = tfidf_matrix.todense()

# Преобразование в DataFrame для удобства
import pandas as pd

df_tfidf = pd.DataFrame(dense, columns=feature_names)

# Вывод результата
print(df_tfidf)

    вместе   играют    ковре     кошка     лежит     ловит      мышь  \
0  0.00000  0.00000  0.57458  0.366747  0.000000  0.000000  0.000000   
1  0.00000  0.00000  0.00000  0.000000  0.555283  0.000000  0.000000   
2  0.57458  0.57458  0.00000  0.366747  0.000000  0.000000  0.000000   
3  0.00000  0.00000  0.00000  0.411378  0.000000  0.644503  0.644503   

         на      полу    сидит    собака  
0  0.453005  0.000000  0.57458  0.000000  
1  0.437791  0.555283  0.00000  0.437791  
2  0.000000  0.000000  0.00000  0.453005  
3  0.000000  0.000000  0.00000  0.000000  


### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [69]:
from typing import Iterator
import numpy as np

n = 4
mat = np.ones((5, n))
vec1 = np.random.rand(n // 2)
vec2 = np.random.rand(n - n // 2)

def MAP(coordinates:(int, int), value:int):
    i, j = coordinates
    if (j < len(vec1)):
        yield (i, value * vec1[j])
    else:
        yield (i, value * vec2[j - len(vec1)])

def REDUCE(i:int, products:Iterator[NamedTuple]):
    sum = 0
    for p in products:
        sum += p
    yield (i, sum)

def RECORDREADER():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield ((i, j), mat[i, j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

TypeError: unsupported operand type(s) for +=: 'int' and 'tuple'

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
  (j, k) = k1
  for i in range(small_mat.shape[0]):
    yield ((i, k), small_mat[i][j] * v1)
  
def REDUCE(key, values):
  yield (key, sum(values))

Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for i in range (small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield (((i, j), small_mat[i,j]), ((j, k), big_mat[j,k]))

def MAP(mat1, mat2):
  (k1, v1) = mat1
  (k2, v2) = mat2
  (i, _) = k1
  (_, k) = k2
  yield ((i, k), v1 * v2)

def REDUCE(key, values):
  yield (key, sum(values))

# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution))  # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [ ]:
maps = 2
reducers = 2

def INPUTFORMAT():
    global maps
    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))

def MAP(mat1, mat2):
  (k1, v1) = mat1
  (k2, v2) = mat2
  (i, _) = k1
  (_, k) = k2
  yield ((i, k), v1 * v2)

def REDUCE(key, values):
  yield (key, sum(values))

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]
print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[0.72545556 0.29093086 0.77805527 0.90401892 0.55751217 0.47966874
  0.71345769 0.5892741  0.59739361 0.31073672 0.61941075 0.49142408
  0.57513451 0.68445219 0.18337485 0.38420941 0.62708168 0.03204539
  0.57846168 0.76240317 0.47545268 0.87025651 0.30519063 0.40590073
  0.32308738 0.46932484 0.894175   0.8460157  1.10831175 0.8201426
  0.58219464 0.28890822 0.58578834 0.2977747  0.82241266 0.58307981
  0.61491737 0.85685444 0.79324807 0.30491726]
 [1.18371409 0.58700665 1.19464486 1.10533862 0.90340918 0.83411431
  1.25765501 0.92529937 1.08984849 0.7641095  0.65109063 0.93541797
  0.89034009 0.9018966  0.21043347 0.42576792 0.56462363 0.05567379
  0.73393152 0.92373854 0.7872857  1.28569273 0.63450758 0.67851092
  0.43324249 0.59194539 1.38359912 1.02932682 1.59490772 0.99119203
  0.84330976 0.66174044 1.31839131 0.3490198  1.07947247 0.96456363
  1.12550273 1.13797901 1.35062342 0.59839567]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
Да, решение будет работать.